In [ ]:
!pip  install pm4py
!pip install python-Levenshtein
!pip install scikit-learn
!pip install paretoset

In [ ]:
import pm4py
import cudf
import cupy as cp
import numpy as np
from sklearn.cluster import DBSCAN as SklearnDBSCAN
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.manifold import TSNE
import seaborn as sns
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.conformance.tokenreplay import algorithm as token_replay
from pm4py.algo.evaluation.precision import algorithm as precision_evaluator
from pm4py.algo.evaluation.generalization import algorithm as generalization_evaluator
from pm4py.objects.petri_net.utils import petri_utils
from collections import Counter
from Levenshtein import distance as levenshtein_distance
import time
import cupyx.scipy.sparse as cusp
import concurrent.futures as cf
import math
from numba import cuda
import matplotlib.pyplot as plt
import pandas as pd
import os

def setup_environment():
    """Install required packages and set up GPU environment."""
    print(f"Running: Using {cp.cuda.runtime.getDeviceCount()} GPU(s) with CUDA 12.2...")

def load_event_log(file_path):
    """Load and process the event log from an XES file, limiting to the first 1000 traces."""
    print("Loading: .xes file...")
    start_time = time.time()
    log = xes_importer.apply(file_path)
    print(f"Completed loading .xes file: {time.time() - start_time:.2f}s")
    
    print("Converting: Log to list of traces (limited to first 1000 traces)...")
    # Limit to the first 1000 traces
    limited_log = log
    traces = [tuple(event["concept:name"] for event in trace) for trace in limited_log]
    unique_activities = sorted(set().union(*[set(trace) for trace in traces]))
    n_traces = len(traces)
    print(f"Completed log conversion: {time.time() - start_time:.2f}s (Number of traces: {n_traces})")
    return limited_log, traces, unique_activities, n_traces

def compute_quality_metrics():
    """Define functions for computing quality metrics."""
    def compute_fitness(net, initial_marking, final_marking, trace_log):
        replayed_traces = token_replay.apply(trace_log, net, initial_marking, final_marking)
        fitness = sum(t["trace_fitness"] for t in replayed_traces) / len(replayed_traces)
        return fitness

    def compute_simplicity(net):
        num_transitions = len(net.transitions)
        num_places = len(net.places)
        return num_transitions + num_places

    def compute_precision(net, initial_marking, final_marking, trace_log):
        precision = precision_evaluator.apply(
            trace_log, net, initial_marking, final_marking,
            variant=precision_evaluator.Variants.ETCONFORMANCE_TOKEN
        )
        return precision

    def compute_generalization(net, initial_marking, final_marking, trace_log):
        generalization = generalization_evaluator.apply(trace_log, net, initial_marking, final_marking)
        return generalization

    def compute_silhouette_index(data, labels, metric='euclidean', use_gpu=True):
        print("Computing: Running silhouette index...")
        start_time = time.time()
        data = cp.asarray(data) if use_gpu else np.asarray(data)
        labels = cp.asarray(labels) if use_gpu else np.asarray(labels)
        
        unique_labels = cp.unique(labels)
        n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)
        
        if n_clusters < 2:
            print("Warning: Need at least 2 clusters for meaningful Silhouette Index.")
            return -1
        
        dist_matrix = data if metric == 'precomputed' else pairwise_distances(
            data.get() if use_gpu else data, metric=metric
        )
        dist_matrix = cp.asarray(dist_matrix) if use_gpu else dist_matrix
        
        silhouette_scores = []
        for i in range(len(labels)):
            if labels[i] == -1:
                continue
            same_cluster = (labels == labels[i])
            a_x = cp.mean(dist_matrix[i, same_cluster]) if cp.sum(same_cluster) > 1 else 0
            b_x = cp.inf
            for label in unique_labels:
                if label == labels[i] or label == -1:
                    continue
                other_cluster = (labels == label)
                if cp.sum(other_cluster) > 0:
                    b_x = min(b_x, cp.mean(dist_matrix[i, other_cluster]))
            s_x = 0 if a_x == 0 and b_x == 0 else (b_x - a_x) / max(a_x, b_x)
            silhouette_scores.append(float(s_x.get() if use_gpu else s_x))
        
        silhouette_avg = np.mean(silhouette_scores) if silhouette_scores else -1
        print(f"Completed Silhouette Index: {silhouette_avg:.4f} (Time: {time.time() - start_time:.2f}s)")
        return silhouette_avg
    
    return {
        "fitness": compute_fitness,
        "simplicity": compute_simplicity,
        "precision": compute_precision,
        "generalization": compute_generalization,
        "silhouette": compute_silhouette_index
    }

def compute_feature_vectors(traces, unique_activities, n_traces):
    """Compute feature vectors for different methods."""
    def bag_of_activities():
        print("Running: Computing vectors for Bag-of-activities (A1)...")
        start_time = time.time()
        vectors = cp.zeros((n_traces, len(unique_activities)))
        for i, trace in enumerate(traces):
            counter = Counter(trace)
            for j, activity in enumerate(unique_activities):
                vectors[i, j] = counter[activity]
        print(f"Completed A1: {time.time() - start_time:.2f}s")
        return vectors, "A1"

    def k_gram_model(k=3):
        print("Running: Computing vectors for k-gram model (A2)...")
        start_time = time.time()
        k_grams_set = set()
        for trace in traces:
            for i in range(len(trace) - k + 1):
                k_grams_set.add(trace[i:i+k])
        k_grams_list = sorted(k_grams_set)
        vectors = cp.zeros((n_traces, len(k_grams_list)))
        for i, trace in enumerate(traces):
            trace_k_grams = Counter([trace[j:j+k] for j in range(len(trace) - k + 1)])
            for j, k_gram in enumerate(k_grams_list):
                vectors[i, j] = trace_k_grams[k_gram]
        print(f"Completed A2: {time.time() - start_time:.2f}s")
        return vectors, "A2"

    @cuda.jit
    def levenshtein_kernel(s_batch, t_batch, distances):
        i = cuda.grid(1)
        if i < s_batch.shape[0] * t_batch.shape[0]:
            s_idx = i // t_batch.shape[0]
            t_idx = i % t_batch.shape[0]
            s_len = int(s_batch[s_idx, 0])
            t_len = int(t_batch[t_idx, 0])
            s = s_batch[s_idx, 1:s_len+1]
            t = t_batch[t_idx, 1:t_len+1]
            dp = cuda.local.array((100, 100), dtype=cp.float32)
            for m in range(s_len + 1):
                for n in range(t_len + 1):
                    if m == 0 and n == 0:
                        dp[m, n] = 0
                    elif m == 0:
                        dp[m, n] = dp[m, n-1] + 1
                    elif n == 0:
                        dp[m, n] = dp[m-1, n] + 1
                    else:
                        sub_cost = 0 if s[m-1] == t[n-1] else 1
                        dp[m, n] = min(dp[m-1, n-1] + sub_cost, dp[m-1, n] + 1, dp[m, n-1] + 1)
            distances[s_idx, t_idx] = dp[s_len, t_len]

    def levenshtein_distance_matrix(chunk_size=10000):
        print("Running: Computing Levenshtein distance matrix on multi-GPU (A3)...")
        start_time = time.time()
        n_chunks = math.ceil(n_traces / chunk_size)
        max_len = max(max(len(t) for t in traces), 100)
        act_to_idx = {act: idx for idx, act in enumerate(unique_activities)}
        traces_padded = cp.zeros((n_traces, max_len + 1), dtype=cp.float32)
        for i, trace in enumerate(traces):
            traces_padded[i, 0] = len(trace)
            for j, act in enumerate(trace):
                traces_padded[i, j + 1] = act_to_idx[act]

        def process_chunk(i, gpu_id):
            with cp.cuda.Device(gpu_id):
                start_idx = i * chunk_size
                end_idx = min((i + 1) * chunk_size, n_traces)
                s_batch = traces_padded[start_idx:end_idx]
                t_batch = traces_padded
                distances = cp.zeros((s_batch.shape[0], t_batch.shape[0]), dtype=cp.float32)
                threads_per_block = 256
                blocks_per_grid = math.ceil((s_batch.shape[0] * t_batch.shape[0]) / threads_per_block)
                levenshtein_kernel[blocks_per_grid, threads_per_block](
                    s_batch, t_batch, distances
                )
                return distances

        dist_matrix_chunks = []
        n_gpus = cp.cuda.runtime.getDeviceCount()
        with cf.ThreadPoolExecutor(max_workers=n_gpus) as executor:
            futures = [executor.submit(process_chunk, i, i % n_gpus) for i in range(n_chunks)]
            for idx, future in enumerate(futures):
                dist_matrix_chunks.append(future.result())
                progress = (idx + 1) / n_chunks
                eta = (time.time() - start_time) / progress * (1 - progress)
                print(f"A3 Distance Matrix: Processed {idx+1}/{n_chunks} chunks ({progress*100:.2f}%), ETA: {eta:.2f}s")

        dist_matrix = cp.vstack(dist_matrix_chunks)
        dist_matrix_full = cp.zeros((n_traces, n_traces), dtype=cp.float32)
        dist_matrix_full[:dist_matrix.shape[0], :] = dist_matrix
        dist_matrix_full.T[:dist_matrix.shape[0], :] = dist_matrix
        dist_matrix_full = cp.maximum(dist_matrix_full, 0)
        nnz = cusp.csr_matrix(dist_matrix_full).nnz
        print(f"Completed A3: {time.time() - start_time:.2f}s")
        print(f"A3 Distance Matrix: Non-zero elements = {nnz}, Total elements = {n_traces * n_traces}")
        return dist_matrix_full, "A3"

    def get_3grams():
        activity_to_idx = {a: i for i, a in enumerate(unique_activities)}
        g3_counts = cp.zeros((len(unique_activities), len(unique_activities), len(unique_activities)), dtype=cp.int32)
        for trace in traces:
            for i in range(len(trace) - 2):
                x, a, y = trace[i:i+3]
                g3_counts[activity_to_idx[x], activity_to_idx[a], activity_to_idx[y]] += 1
        return g3_counts

    def compute_substitution_scores():
        print("Running: Computing Substitution Scores (Algorithm 1)...")
        start_time = time.time()
        n_activities = len(unique_activities)
        g3_counts = get_3grams()
        Xa = [set() for _ in range(n_activities)]
        for x_idx in range(n_activities):
            for a_idx in range(n_activities):
                for y_idx in range(n_activities):
                    if g3_counts[x_idx, a_idx, y_idx] > 0:
                        Xa[a_idx].add((x_idx, y_idx))
        C = cp.zeros((n_activities, n_activities), dtype=cp.float32)
        for a_idx in range(n_activities):
            for b_idx in range(n_activities):
                if a_idx == b_idx:
                    counts = g3_counts[:, a_idx, :]
                    C[a_idx, a_idx] = cp.sum(counts * (counts - 1) / 2)
                else:
                    Xab = Xa[a_idx] & Xa[b_idx]
                    for x_idx, y_idx in Xab:
                        C[a_idx, b_idx] += g3_counts[x_idx, a_idx, y_idx] * g3_counts[x_idx, b_idx, y_idx]
        NC = cp.sum(C)
        M = C / NC
        pa = cp.zeros(n_activities)
        for a_idx in range(n_activities):
            pa[a_idx] = M[a_idx, a_idx] + cp.sum(M[a_idx, :]) - M[a_idx, a_idx]
        E = cp.zeros((n_activities, n_activities))
        for a_idx in range(n_activities):
            for b_idx in range(n_activities):
                E[a_idx, b_idx] = pa[a_idx]**2 if a_idx == b_idx else 2 * pa[a_idx] * pa[b_idx]
        S = cp.log2(M / E)
        S = cp.where(cp.isinf(S) | cp.isnan(S), 0, S)
        print(f"Completed Substitution Scores: {time.time() - start_time:.2f}s")
        print(f"sub_scores: min={cp.min(S):.4f}, max={cp.max(S):.4f}, mean={cp.mean(S):.4f}")
        return S

    def compute_insertion_scores():
        print("Running: Computing Insertion Scores (Algorithm 2)...")
        start_time = time.time()
        n_activities = len(unique_activities)
        g3_counts = get_3grams()
        Xa = [set() for _ in range(n_activities)]
        for x_idx in range(n_activities):
            for a_idx in range(n_activities):
                for y_idx in range(n_activities):
                    if g3_counts[x_idx, a_idx, y_idx] > 0:
                        Xa[a_idx].add((x_idx, y_idx))
        count_right = cp.zeros((n_activities, n_activities), dtype=cp.float32)
        for a_idx in range(n_activities):
            for x_idx in range(n_activities):
                count_right[a_idx, x_idx] = cp.sum(g3_counts[x_idx, a_idx, :])
        norm = cp.sum(count_right, axis=1)
        pa = norm / cp.sum(norm)
        norm_count_right = count_right / norm[:, cp.newaxis]
        norm_count_right = cp.where(cp.isnan(norm_count_right), 0, norm_count_right)
        ins_scores = cp.log2(norm_count_right / (pa[:, cp.newaxis] * pa[cp.newaxis, :]))
        ins_scores = cp.where(cp.isinf(ins_scores) | cp.isnan(ins_scores), 0, ins_scores)
        print(f"Completed Insertion Scores: {time.time() - start_time:.2f}s")
        print(f"ins_scores: min={cp.min(ins_scores):.4f}, max={cp.max(ins_scores):.4f}, mean={cp.mean(ins_scores):.4f}")
        return ins_scores

    @cuda.jit
    def edit_distance_kernel(s_batch, t_batch, sub_scores, ins_scores, distances):
        i = cuda.grid(1)
        if i < s_batch.shape[0] * t_batch.shape[0]:
            s_idx = i // t_batch.shape[0]
            t_idx = i % t_batch.shape[0]
            s_len = s_batch[s_idx, 0]
            t_len = t_batch[t_idx, 0]
            s = s_batch[s_idx, 1:int(s_len+1)]
            t = t_batch[t_idx, 1:int(t_len+1)]
            dp = cuda.local.array((100, 100), dtype=cp.float32)
            for m in range(int(s_len) + 1):
                for n in range(int(t_len) + 1):
                    if m == 0 and n == 0:
                        dp[m, n] = 0
                    elif m == 0:
                        dp[m, n] = dp[m, n-1] + max(1.0, abs(ins_scores[int(t[n-1]), int(t[n-1])]))
                    elif n == 0:
                        dp[m, n] = dp[m-1, n] + max(1.0, abs(ins_scores[int(s[m-1]), int(s[m-1])]))
                    else:
                        sub_cost = 0 if s[m-1] == t[n-1] else max(1.0, abs(sub_scores[int(s[m-1]), int(t[n-1])]))
                        del_cost = max(1.0, abs(ins_scores[int(s[m-1]), int(s[m-1])]))
                        ins_cost = max(1.0, abs(ins_scores[int(t[n-1]), int(t[n-1])]))
                        dp[m, n] = min(dp[m-1, n-1] + sub_cost, dp[m-1, n] + del_cost, dp[m, n-1] + ins_cost)
            distances[s_idx, t_idx] = dp[int(s_len), int(t_len)]

    def generic_edit_distance_matrix(sub_scores, ins_scores, chunk_size=10000):
        print("Running: Computing Generic Edit Distance matrix on multi-GPU (A4)...")
        start_time = time.time()
        n_chunks = math.ceil(n_traces / chunk_size)
        max_len = max(max(len(t) for t in traces), 100)
        act_to_idx = {act: idx for idx, act in enumerate(unique_activities)}
        traces_padded = cp.zeros((n_traces, max_len + 1), dtype=cp.float32)
        for i, trace in enumerate(traces):
            traces_padded[i, 0] = len(trace)
            for j, act in enumerate(trace):
                traces_padded[i, j + 1] = act_to_idx[act]
        sub_scores_gpu = cp.asarray(sub_scores)
        ins_scores_gpu = cp.asarray(ins_scores)

        def process_chunk(i, gpu_id):
            with cp.cuda.Device(gpu_id):
                start_idx = i * chunk_size
                end_idx = min((i + 1) * chunk_size, n_traces)
                s_batch = traces_padded[start_idx:end_idx]
                t_batch = traces_padded
                distances = cp.zeros((s_batch.shape[0], t_batch.shape[0]), dtype=cp.float32)
                threads_per_block = 256
                blocks_per_grid = math.ceil((s_batch.shape[0] * t_batch.shape[0]) / threads_per_block)
                edit_distance_kernel[blocks_per_grid, threads_per_block](
                    s_batch, t_batch, sub_scores_gpu, ins_scores_gpu, distances
                )
                return distances

        dist_matrix_chunks = []
        n_gpus = cp.cuda.runtime.getDeviceCount()
        with cf.ThreadPoolExecutor(max_workers=n_gpus) as executor:
            futures = [executor.submit(process_chunk, i, i % n_gpus) for i in range(n_chunks)]
            for idx, future in enumerate(futures):
                dist_matrix_chunks.append(future.result())
                progress = (idx + 1) / n_chunks
                eta = (time.time() - start_time) / progress * (1 - progress)
                print(f"A4 Distance Matrix: Processed {idx+1}/{n_chunks} chunks ({progress*100:.2f}%), ETA: {eta:.2f}s")
        
        dist_matrix = cp.vstack(dist_matrix_chunks)
        dist_matrix_full = cp.zeros((n_traces, n_traces), dtype=cp.float32)
        dist_matrix_full[:dist_matrix.shape[0], :] = dist_matrix
        dist_matrix_full.T[:dist_matrix.shape[0], :] = dist_matrix
        dist_matrix_full = cp.maximum(dist_matrix_full, 0)
        print(f"Completed Distance Matrix: {time.time() - start_time:.2f}s")
        nnz = cusp.csr_matrix(dist_matrix_full).nnz
        print(f"A4 Distance Matrix: Non-zero elements = {nnz}, Total elements = {n_traces * n_traces}")
        return dist_matrix_full, "A4"

    @cuda.jit
    def equivalence_kernel(traces_array, counts, R_eq):
        t_idx, a_idx, b_idx = cuda.grid(3)
        if t_idx < traces_array.shape[0] and a_idx < R_eq.shape[0] and b_idx < R_eq.shape[1]:
            if a_idx == b_idx:
                cuda.atomic.add(R_eq, (a_idx, b_idx), 1)
            else:
                trace_len = int(traces_array[t_idx, 0])
                count_a = 0
                count_b = 0
                for i in range(trace_len):
                    if int(traces_array[t_idx, i + 1]) == a_idx:
                        count_a += 1
                    if int(traces_array[t_idx, i + 1]) == b_idx:
                        count_b += 1
                if count_a == count_b:
                    cuda.atomic.add(R_eq, (a_idx, b_idx), 1)

    @cuda.jit
    def always_after_kernel(traces_array, R_aa):
        t_idx, a_idx, b_idx = cuda.grid(3)
        if t_idx < traces_array.shape[0] and a_idx < R_aa.shape[0] and b_idx < R_aa.shape[1] and a_idx != b_idx:
            trace_len = int(traces_array[t_idx, 0])
            idx_a = cuda.local.array(100, dtype=cp.int32)
            idx_b = cuda.local.array(100, dtype=cp.int32)
            a_count = 0
            b_count = 0
            for i in range(trace_len):
                if int(traces_array[t_idx, i + 1]) == a_idx and a_count < 100:
                    idx_a[a_count] = i
                    a_count += 1
                if int(traces_array[t_idx, i + 1]) == b_idx and b_count < 100:
                    idx_b[b_count] = i
                    b_count += 1
            if a_count > 0 and b_count > 0:
                if idx_b[0] > idx_a[a_count - 1]:
                    cuda.atomic.add(R_aa, (a_idx, b_idx), 1)

    @cuda.jit
    def always_before_kernel(traces_array, R_ab):
        t_idx, a_idx, b_idx = cuda.grid(3)
        if t_idx < traces_array.shape[0] and a_idx < R_ab.shape[0] and b_idx < R_ab.shape[1] and a_idx != b_idx:
            trace_len = int(traces_array[t_idx, 0])
            idx_a = cuda.local.array(100, dtype=cp.int32)
            idx_b = cuda.local.array(100, dtype=cp.int32)
            a_count = 0
            b_count = 0
            for i in range(trace_len):
                if int(traces_array[t_idx, i + 1]) == a_idx and a_count < 100:
                    idx_a[a_count] = i
                    a_count += 1
                if int(traces_array[t_idx, i + 1]) == b_idx and b_count < 100:
                    idx_b[b_count] = i
                    b_count += 1
            if a_count > 0 and b_count > 0:
                if idx_a[a_count - 1] < idx_b[0]:
                    cuda.atomic.add(R_ab, (a_idx, b_idx), 1)

    @cuda.jit
    def never_together_kernel(traces_array, R_nt):
        t_idx, a_idx, b_idx = cuda.grid(3)
        if t_idx < traces_array.shape[0] and a_idx < R_nt.shape[0] and b_idx < R_nt.shape[1] and a_idx < b_idx:
            trace_len = int(traces_array[t_idx, 0])
            has_a = False
            has_b = False
            for i in range(trace_len):
                if int(traces_array[t_idx, i + 1]) == a_idx:
                    has_a = True
                if int(traces_array[t_idx, i + 1]) == b_idx:
                    has_b = True
            if not (has_a and has_b):
                cuda.atomic.add(R_nt, (a_idx, b_idx), 1)
                cuda.atomic.add(R_nt, (b_idx, a_idx), 1)

    @cuda.jit
    def directly_follows_kernel(traces_array, R_df):
        t_idx = cuda.grid(1)
        if t_idx < traces_array.shape[0]:
            trace_len = int(traces_array[t_idx, 0])
            for i in range(trace_len - 1):
                a_idx = int(traces_array[t_idx, i + 1])
                b_idx = int(traces_array[t_idx, i + 2])
                cuda.atomic.add(R_df, (a_idx, b_idx), 1)

    @cuda.jit
    def log_skeleton_distance_kernel(s_batch, t_batch, R_eq, R_aa, R_ab, R_nt, R_df, C_min, C_max, distances):
        i = cuda.grid(1)
        if i < s_batch.shape[0] * t_batch.shape[0]:
            s_idx = i // t_batch.shape[0]
            t_idx = i % t_batch.shape[0]
            s_len = int(s_batch[s_idx, 0])
            t_len = int(t_batch[t_idx, 0])
            s = s_batch[s_idx, 1:s_len+1]
            t = t_batch[t_idx, 1:t_len+1]
            distance = 0.0

            # R_eq
            for a_idx in range(R_eq.shape[0]):
                for b_idx in range(a_idx, R_eq.shape[1]):
                    if R_eq[a_idx, b_idx]:
                        count_s_a = 0
                        count_s_b = 0
                        count_t_a = 0
                        count_t_b = 0
                        for m in range(s_len):
                            if int(s[m]) == a_idx:
                                count_s_a += 1
                            if int(s[m]) == b_idx:
                                count_s_b += 1
                        for n in range(t_len):
                            if int(t[n]) == a_idx:
                                count_t_a += 1
                            if int(t[n]) == b_idx:
                                count_t_b += 1
                        if (count_s_a == count_s_b) != (count_t_a == count_t_b):
                            distance += 1.0

            # R_aa and R_ab (combined to reduce memory access)
            for a_idx in range(R_aa.shape[0]):
                for b_idx in range(R_aa.shape[1]):
                    if a_idx == b_idx:
                        continue
                    idx_s_a = cuda.local.array(50, dtype=cp.int32)
                    idx_s_b = cuda.local.array(50, dtype=cp.int32)
                    idx_t_a = cuda.local.array(50, dtype=cp.int32)
                    idx_t_b = cuda.local.array(50, dtype=cp.int32)
                    s_a_count = 0
                    s_b_count = 0
                    t_a_count = 0
                    t_b_count = 0
                    for m in range(s_len):
                        if int(s[m]) == a_idx and s_a_count < 50:
                            idx_s_a[s_a_count] = m
                            s_a_count += 1
                        if int(s[m]) == b_idx and s_b_count < 50:
                            idx_s_b[s_b_count] = m
                            s_b_count += 1
                    for n in range(t_len):
                        if int(t[n]) == a_idx and t_a_count < 50:
                            idx_t_a[t_a_count] = n
                            t_a_count += 1
                        if int(t[n]) == b_idx and t_b_count < 50:
                            idx_t_b[t_b_count] = n
                            t_b_count += 1
                    # R_aa
                    if R_aa[a_idx, b_idx]:
                        after_s = s_a_count == 0 or s_b_count == 0 or idx_s_b[0] > idx_s_a[s_a_count - 1]
                        after_t = t_a_count == 0 or t_b_count == 0 or idx_t_b[0] > idx_t_a[t_a_count - 1]
                        if after_s != after_t:
                            distance += 1.0
                    # R_ab
                    if R_ab[a_idx, b_idx]:
                        before_s = s_a_count == 0 or s_b_count == 0 or idx_s_a[s_a_count - 1] < idx_s_b[0]
                        before_t = t_a_count == 0 or t_b_count == 0 or idx_t_a[t_a_count - 1] < idx_t_b[0]
                        if before_s != before_t:
                            distance += 1.0

            # R_nt
            for a_idx in range(R_nt.shape[0]):
                for b_idx in range(a_idx + 1, R_nt.shape[1]):
                    if R_nt[a_idx, b_idx]:
                        has_s_a = False
                        has_s_b = False
                        has_t_a = False
                        has_t_b = False
                        for m in range(s_len):
                            if int(s[m]) == a_idx:
                                has_s_a = True
                            if int(s[m]) == b_idx:
                                has_s_b = True
                        for n in range(t_len):
                            if int(t[n]) == a_idx:
                                has_t_a = True
                            if int(t[n]) == b_idx:
                                has_t_b = True
                        if (has_s_a and has_s_b) != (has_t_a and has_t_b):
                            distance += 1.0

            # R_df
            for a_idx in range(R_df.shape[0]):
                for b_idx in range(R_df.shape[1]):
                    if R_df[a_idx, b_idx] > 0:
                        df_s = 0
                        df_t = 0
                        for m in range(s_len - 1):
                            if int(s[m]) == a_idx and int(s[m + 1]) == b_idx:
                                df_s += 1
                        for n in range(t_len - 1):
                            if int(t[n]) == a_idx and int(t[n + 1]) == b_idx:
                                df_t += 1
                        distance += abs(df_s - df_t) * 0.1

            # Counters
            for a_idx in range(len(C_min)):
                count_s = 0
                count_t = 0
                for m in range(s_len):
                    if int(s[m]) == a_idx:
                        count_s += 1
                for n in range(t_len):
                    if int(t[n]) == a_idx:
                        count_t += 1
                if not (C_min[a_idx] <= count_s <= C_max[a_idx] and C_min[a_idx] <= count_t <= C_max[a_idx]):
                    distance += abs(count_s - count_t) * 0.1

            distances[s_idx, t_idx] = distance

    def log_skeleton_distance_matrix(chunk_size=None):
        print("Running: Computing Log Skeleton distance matrix on multi-GPU (A5)...")
        start_time = time.time()

        # Step 1: Preprocess traces
        alpha, omega = "α", "ω"
        extended_activities = [alpha] + unique_activities + [omega]
        act_to_idx = {act: idx for idx, act in enumerate(extended_activities)}
        extended_traces = [[alpha] + list(trace) + [omega] for trace in traces]
        n_activities = len(extended_activities)
        max_len = max(max(len(t) for t in extended_traces), 100)
        if max_len > 100:
            raise ValueError(f"Trace length {max_len} exceeds maximum supported length (100). Increase kernel array size.")

        traces_array = cp.zeros((n_traces, max_len + 1), dtype=cp.int32)
        for i, trace in enumerate(extended_traces):
            traces_array[i, 0] = len(trace)
            for j, act in enumerate(trace):
                traces_array[i, j + 1] = act_to_idx[act]

        # Step 2: Compute relations on GPU
        print("Computing: Log Skeleton relations...")
        rel_start_time = time.time()

        # Dynamic chunk size based on GPU memory
        if chunk_size is None:
            gpu_mem = cp.cuda.runtime.memGetInfo()[0] // (1024 ** 2)  # Free memory in MB
            est_mem_per_trace = max_len * 4 / (1024 ** 2)  # Approx memory per trace in MB
            chunk_size = max(1000, int(gpu_mem * 0.5 / est_mem_per_trace))
            print(f"Dynamic chunk_size set to {chunk_size} based on GPU memory ({gpu_mem} MB free)")

        # Equivalence relation (R_eq)
        R_eq = cp.zeros((n_activities, n_activities), dtype=cp.int32)
        threads_per_block = (8, 8, 8)
        blocks_per_grid = (
            math.ceil(n_traces / threads_per_block[0]),
            math.ceil(n_activities / threads_per_block[1]),
            math.ceil(n_activities / threads_per_block[2])
        )
        equivalence_kernel[blocks_per_grid, threads_per_block](traces_array, cp.zeros((n_traces, n_activities), dtype=cp.int32), R_eq)
        R_eq = (R_eq == n_traces).astype(cp.bool_)
        R_eq |= R_eq.T

        # Always-after relation (R_aa)
        R_aa = cp.zeros((n_activities, n_activities), dtype=cp.int32)
        always_after_kernel[blocks_per_grid, threads_per_block](traces_array, R_aa)
        R_aa = (R_aa == n_traces).astype(cp.bool_)

        # Always-before relation (R_ab)
        R_ab = cp.zeros((n_activities, n_activities), dtype=cp.int32)
        always_before_kernel[blocks_per_grid, threads_per_block](traces_array, R_ab)
        R_ab = (R_ab == n_traces).astype(cp.bool_)

        # Never-together relation (R_nt)
        R_nt = cp.zeros((n_activities, n_activities), dtype=cp.int32)
        never_together_kernel[blocks_per_grid, threads_per_block](traces_array, R_nt)
        R_nt = (R_nt == n_traces).astype(cp.bool_)

        # Directly-follows relation (R_df)
        R_df = cp.zeros((n_activities, n_activities), dtype=cp.int32)
        threads_per_block_df = 256
        blocks_per_grid_df = math.ceil(n_traces / threads_per_block_df)
        directly_follows_kernel[blocks_per_grid_df, threads_per_block_df](traces_array, R_df)

        # Counters
        counts = cp.zeros((n_traces, n_activities), dtype=cp.int32)
        for i in range(n_traces):
            trace_len = int(traces_array[i, 0])
            for j in range(trace_len):
                act_idx = int(traces_array[i, j + 1])
                counts[i, act_idx] += 1
        C_min = cp.min(counts, axis=0)
        C_max = cp.max(counts, axis=0)

        print(f"Completed relations computation: {time.time() - rel_start_time:.2f}s")

        # Step 3: Compute distance matrix on GPU
        n_chunks = math.ceil(n_traces / chunk_size)
        traces_padded = traces_array

        def process_chunk(i, gpu_id):
            with cp.cuda.Device(gpu_id):
                start_idx = i * chunk_size
                end_idx = min((i + 1) * chunk_size, n_traces)
                s_batch = traces_padded[start_idx:end_idx]
                t_batch = traces_padded
                distances = cp.zeros((s_batch.shape[0], t_batch.shape[0]), dtype=cp.float32)
                threads_per_block = 256
                blocks_per_grid = math.ceil((s_batch.shape[0] * t_batch.shape[0]) / threads_per_block)
                log_skeleton_distance_kernel[blocks_per_grid, threads_per_block](
                    s_batch, t_batch, R_eq, R_aa, R_ab, R_nt, R_df, C_min, C_max, distances
                )
                return distances

        dist_matrix_chunks = []
        n_gpus = cp.cuda.runtime.getDeviceCount()
        with cf.ThreadPoolExecutor(max_workers=n_gpus) as executor:
            futures = [executor.submit(process_chunk, i, i % n_gpus) for i in range(n_chunks)]
            for idx, future in enumerate(futures):
                dist_matrix_chunks.append(future.result())
                progress = (idx + 1) / n_chunks
                eta = (time.time() - start_time) / progress * (1 - progress)
                print(f"A5 Distance Matrix: Processed {idx+1}/{n_chunks} chunks ({progress*100:.2f}%), ETA: {eta:.2f}s")

        dist_matrix = cp.vstack(dist_matrix_chunks)
        dist_matrix_full = cp.zeros((n_traces, n_traces), dtype=cp.float32)
        dist_matrix_full[:dist_matrix.shape[0], :] = dist_matrix
        dist_matrix_full.T[:dist_matrix.shape[0], :] = dist_matrix
        dist_matrix_full = cp.maximum(dist_matrix_full, 0)
        nnz = cusp.csr_matrix(dist_matrix_full).nnz
        print(f"Completed A5: {time.time() - start_time:.2f}s")
        print(f"A5 Distance Matrix: Non-zero elements = {nnz}, Total elements = {n_traces * n_traces}")
        return dist_matrix_full, "A5"

    sub_scores = compute_substitution_scores()
    ins_scores = compute_insertion_scores()
    return [
        bag_of_activities(),
        k_gram_model(),
        levenshtein_distance_matrix(),
        generic_edit_distance_matrix(sub_scores, ins_scores),
        log_skeleton_distance_matrix()
    ]

def estimate_eps(data):
    """Estimate eps parameter for DBSCAN."""
    if hasattr(data, 'toarray'):
        dist_array = data.toarray().get()
    elif hasattr(data, 'get'):
        dist_array = data.get()
    else:
        dist_array = cp.pdist(data).get()
    positive_dists = dist_array[dist_array > 0]
    return np.mean(positive_dists) * 0.05 if len(positive_dists) > 0 else 0.1

def perform_clustering_and_evaluation(log, traces, methods, metrics):
    """Perform clustering and evaluate results."""
    results = {}
    tsne_data = {}
    labels_data = {}
    
    for method, name in methods:
        print(f"Running: Clustering and evaluation for {name}...")
        start_time = time.time()
        
        eps = estimate_eps(method)
        print(f"Estimated eps for {name} (DBSCAN): {eps:.4f}")
        
        if name in ["A1", "A2"]:
            clustering_dbscan = SklearnDBSCAN(eps=eps, min_samples=5)
            labels_dbscan = clustering_dbscan.fit_predict(method.get())
            data_for_kmeans = method.get()
        else:
            dist_matrix = cp.maximum(method, 0)
            clustering_dbscan = SklearnDBSCAN(eps=eps, min_samples=5, metric="precomputed")
            labels_dbscan = clustering_dbscan.fit_predict(dist_matrix.get())
            data_for_kmeans = TSNE(n_components=2, metric="precomputed", init="random", random_state=42).fit_transform(dist_matrix.get())
        
        unique_labels_dbscan = np.unique(labels_dbscan)
        n_clusters_dbscan = len(unique_labels_dbscan) - (1 if -1 in unique_labels_dbscan else 0)
        print(f"Number of clusters found for {name} (DBSCAN): {n_clusters_dbscan}")
        
        labels_kmeans = labels_dbscan
        labels_hierarchical = labels_dbscan
        if n_clusters_dbscan > 1:
            kmeans = KMeans(n_clusters=n_clusters_dbscan, random_state=42)
            labels_kmeans = kmeans.fit_predict(data_for_kmeans)
            hierarchical = AgglomerativeClustering(n_clusters=n_clusters_dbscan)
            labels_hierarchical = hierarchical.fit_predict(data_for_kmeans)
        else:
            print(f"Warning: {name} - DBSCAN found {n_clusters_dbscan} clusters, skipping K-means and Hierarchical.")
        
        silhouette_dbscan = metrics["silhouette"](method if name in ["A1", "A2"] else dist_matrix, 
                                                labels_dbscan, 
                                                metric='euclidean' if name in ["A1", "A2"] else 'precomputed')
        silhouette_kmeans = metrics["silhouette"](data_for_kmeans, labels_kmeans, metric='euclidean') if n_clusters_dbscan > 1 else -1
        silhouette_hierarchical = metrics["silhouette"](data_for_kmeans, labels_hierarchical, metric='euclidean') if n_clusters_dbscan > 1 else -1
        
        print(f"Silhouette Index for {name} - DBSCAN: {silhouette_dbscan:.4f}")
        print(f"Silhouette Index for {name} - K-means: {silhouette_kmeans:.4f}")
        print(f"Silhouette Index for {name} - Hierarchical: {silhouette_hierarchical:.4f}")
        
        tsne = TSNE(n_components=2, random_state=42)
        tsne_result = tsne.fit_transform(data_for_kmeans)
        tsne_data[name] = tsne_result
        labels_data[name] = {
            "K-means": labels_kmeans,
            "Hierarchical": labels_hierarchical,
            "DBSCAN": labels_dbscan
        }
        
        for algo, labels in [("DBSCAN", labels_dbscan), ("K-means", labels_kmeans), ("Hierarchical", labels_hierarchical)]:
            print(f"Running: Clustering with {algo} for {name}...")
            unique_labels = np.unique(labels)
            n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)
            print(f"Number of clusters found for {name} ({algo}): {n_clusters}")
            
            clustered_logs = [pm4py.objects.log.obj.EventLog([trace for trace, label in zip(log, labels) if label == i])
                             for i in unique_labels if i != -1]
            
            fitness_scores = []
            simplicity_scores = []
            precision_scores = []
            generalization_scores = []
            
            for i, cluster_log in enumerate(clustered_logs):
                if len(cluster_log) == 0:
                    continue
                print(f"Running: Mining and evaluating cluster {i+1}/{len(clustered_logs)} for {name} ({algo})...")
                cluster_start = time.time()
                net, im, fm = alpha_miner.apply(cluster_log)
                fitness_scores.append(metrics["fitness"](net, im, fm, cluster_log))
                simplicity_scores.append(metrics["simplicity"](net))
                precision_scores.append(metrics["precision"](net, im, fm, cluster_log))
                generalization_scores.append(metrics["generalization"](net, im, fm, cluster_log))
                
                progress = (i + 1) / len(clustered_logs)
                eta = (time.time() - cluster_start) / progress * (1 - progress)
                print(f"Cluster {i+1}/{len(clustered_logs)} completed: {time.time() - cluster_start:.2f}s, ETA: {eta:.2f}s")
            
            results[f"{name}_{algo}"] = {
                "fitness": np.mean(fitness_scores) if fitness_scores else 0,
                "simplicity": np.mean(simplicity_scores) if simplicity_scores else 0,
                "precision": np.mean(precision_scores) if precision_scores else 0,
                "generalization": np.mean(generalization_scores) if generalization_scores else 0,
                "n_clusters": n_clusters,
                "silhouette": silhouette_dbscan if algo == "DBSCAN" else (silhouette_kmeans if algo == "K-means" else silhouette_hierarchical),
                "time": time.time() - start_time
            }
    
    return results, tsne_data, labels_data

def visualize_results(results, tsne_data, labels_data):
    """Visualize clustering results and display summary table."""
    print("Creating: Summary table...")
    summary_data = []
    for method, metrics in results.items():
        method_name, algo_name = method.split('_')
        summary_data.append({
            'Method': method_name,
            'Algorithm': algo_name,
            'Fitness': metrics['fitness'],
            'Simplicity': metrics['simplicity'],
            'Precision': metrics['precision'],
            'Generalization': metrics['generalization'],
            'Number of Clusters': metrics['n_clusters'],
            'Silhouette Index': metrics['silhouette'],
            'Time (s)': metrics['time']
        })
    
    df = pd.DataFrame(summary_data)
    df = df.round(4)
    df = df.sort_values(by=['Method', 'Algorithm'])
    
    print("\nSummary of Clustering Results:")
    print(df.to_string(index=False))
    
    df.to_csv('BPI_Challenge_2017.csv', index=False)
    print("Summary table saved to 'clustering_results.csv'")
    
    print("\nRunning: Printing detailed results...")
    for method, metrics in results.items():
        print(f"{method}:")
        print(f"  Fitness: {metrics['fitness']:.2f}")
        print(f"  Simplicity: {metrics['simplicity']:.2f}")
        print(f"  Precision: {metrics['precision']:.2f}")
        print(f"  Generalization: {metrics['generalization']:.2f}")
        print(f"  Number of Clusters: {metrics['n_clusters']}")
        print(f"  Silhouette Index: {metrics['silhouette']:.4f}")
        print("    => " + (
            "Good clustering quality (Silhouette > 0.5)" if metrics['silhouette'] > 0.5 else
            "Acceptable clustering quality (0 < Silhouette ≤ 0.5)" if metrics['silhouette'] > 0 else
            "Poor clustering quality (Silhouette ≤ 0)"
        ))
        print(f"  Time: {metrics['time']:.2f}s")

def main(file_path):
    """Main function to run the entire pipeline."""
    setup_environment()
    log, traces, unique_activities, n_traces = load_event_log(file_path)
    metrics = compute_quality_metrics()
    methods = compute_feature_vectors(traces, unique_activities, n_traces)
    results, tsne_data, labels_data = perform_clustering_and_evaluation(log, traces, methods, metrics)
    visualize_results(results, tsne_data, labels_data)
    print("Completed!")

if __name__ == "__main__":
    input_file = "/kaggle/input/event-log/RequestForPayment.xes"
    main(input_file)

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans
from paretoset import paretoset
import warnings
import os

warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

# Disable cudf.pandas acceleration
os.environ["CUDF_PANDAS_ACCELERATE"] = "false"  # Prevent cudf.pandas acceleration

# Print package versions for debugging
print("Package Versions:")
print(f"pandas: {pd.__version__}")
print(f"scikit-learn: {sklearn.__version__}")
print(f"numpy: {np.__version__}")

# Step 1: Load the results
try:
    df = pd.read_csv('BPI_Challenge_2017.csv')
except FileNotFoundError:
    print("Error: 'BPI_Challenge_2017.csv' not found. Ensure the file exists in the working directory.")
    exit(1)

# Convert to standard pandas DataFrame
df = pd.DataFrame(df)

# Inspect data for issues
print("\nData Info:")
print(df.info())
print("\nFirst 5 rows of data:")
print(df.head())
print("\nChecking for NaN or infinite values:")
print(df[['Fitness', 'Simplicity', 'Precision', 'Generalization']].isna().sum())
print(df[['Fitness', 'Simplicity', 'Precision', 'Generalization']].apply(lambda x: np.isinf(x).sum()))

# Step 2: Automatically determine thresholds for Quality classes
def assign_quality_dynamic(silhouette, thresholds):
    if silhouette >= thresholds['good']:
        return 'Good'
    elif silhouette >= thresholds['acceptable']:
        return 'Acceptable'
    else:
        return 'Poor'

# Compute dynamic thresholds using K-means clustering on Silhouette Index
def compute_dynamic_thresholds(silhouette_values):
    silhouette_values = np.array(silhouette_values).reshape(-1, 1)
    if len(np.unique(silhouette_values)) < 3:
        print("Warning: Insufficient unique Silhouette Index values for clustering. Using quantile-based thresholds.")
        return {
            'good': np.percentile(silhouette_values, 75),  # Top 25%
            'acceptable': np.percentile(silhouette_values, 25)  # Bottom 25%
        }
    kmeans = KMeans(n_clusters=3, random_state=42)
    kmeans.fit(silhouette_values)
    cluster_centers = np.sort(kmeans.cluster_centers_.flatten())
    return {
        'good': cluster_centers[2],  # Highest cluster center
        'acceptable': cluster_centers[1]  # Middle cluster center
    }

# Verify Silhouette Index column exists
if 'Silhouette Index' not in df.columns:
    print("Error: 'Silhouette Index' column missing in CSV file.")
    exit(1)

# Compute thresholds and assign Quality
thresholds = compute_dynamic_thresholds(df['Silhouette Index'])
print(f"\nDynamic Thresholds: Good >= {thresholds['good']:.4f}, Acceptable >= {thresholds['acceptable']:.4f}")
df['Quality'] = df['Silhouette Index'].apply(lambda x: assign_quality_dynamic(x, thresholds))
print("\nQuality Distribution:")
print(df['Quality'].value_counts())

# Step 3: Compute composite score for tie-breaking in Pareto Front
def compute_composite_score(row, weights=None):
    if weights is None:
        weights = {'Silhouette Index': 0.4, 'Fitness': 0.3, 'Precision': 0.2, 'Generalization': 0.1}
    # Normalize metrics to [0, 1]
    normalized = {}
    for metric in ['Silhouette Index', 'Fitness', 'Precision', 'Generalization']:
        values = df[metric]
        min_val, max_val = values.min(), values.max()
        normalized[metric] = (row[metric] - min_val) / (max_val - min_val + 1e-10)  # Avoid division by zero
    score = sum(normalized[metric] * weights[metric] for metric in weights)
    return score

df['Combination'] = df['Method'] + '_' + df['Algorithm']
df['Composite_Score'] = df.apply(compute_composite_score, axis=1)

# Step 4: Select Pareto Front and best combination
def select_best_combination(df, thresholds):
    # Filter combinations above acceptable threshold
    good_combinations = df[df['Silhouette Index'] >= thresholds['acceptable']]
    
    if len(good_combinations) == 0:
        print("Warning: No combinations above acceptable threshold. Selecting best by Composite Score.")
        best_combination_idx = df['Composite_Score'].idxmax()
        return best_combination_idx
    
    # Compute Pareto Front
    metrics = ['Silhouette Index', 'Fitness', 'Precision', 'Generalization']
    pareto_mask = paretoset(good_combinations[metrics], sense=['max', 'max', 'max', 'max'])
    pareto_combinations = good_combinations[pareto_mask]
    
    print(f"\nPareto Front: {len(pareto_combinations)} combinations")
    print(pareto_combinations[['Combination', 'Silhouette Index', 'Fitness', 'Precision', 'Generalization', 'Composite_Score']])
    
    if len(pareto_combinations) > 1:
        # Select second-best by Composite Score
        pareto_combinations = pareto_combinations.sort_values(by='Composite_Score', ascending=False)
        best_combination_idx = pareto_combinations.index[1]  # Second-best
    else:
        best_combination_idx = pareto_combinations.index[0]
    
    return best_combination_idx

# Step 5: Check for single class and select best combination if necessary
if len(df['Quality'].unique()) < 2:
    print("Warning: Only one class found in 'Quality'. Skipping classification and selecting best combination by Pareto Front.")
    best_combination_idx = select_best_combination(df, thresholds)
    
    best_combination = df.loc[best_combination_idx, 'Combination']
    best_metrics = df.loc[best_combination_idx, ['Fitness', 'Simplicity', 'Precision', 'Generalization']]
    best_silhouette = df.loc[best_combination_idx, 'Silhouette Index']
    best_composite_score = df.loc[best_combination_idx, 'Composite_Score']
    
    print("\nBest Combination (No Classification):")
    print(f"Combination: {best_combination}")
    print(f"Quality: {df.loc[best_combination_idx, 'Quality']}")
    print(f"Metrics: Fitness={best_metrics['Fitness']:.4f}, Simplicity={best_metrics['Simplicity']:.4f}, "
          f"Precision={best_metrics['Precision']:.4f}, Generalization={best_metrics['Generalization']:.4f}")
    print(f"Silhouette Index: {best_silhouette:.4f}")
    print(f"Composite Score: {best_composite_score:.4f}")
    exit(0)

# Step 6: Preprocess for classification
# Encode categorical variables
le_method = LabelEncoder()
le_algorithm = LabelEncoder()
df['Method_Encoded'] = le_method.fit_transform(df['Method'])
df['Algorithm_Encoded'] = le_algorithm.fit_transform(df['Algorithm'])

# Select features and target
features = ['Fitness', 'Simplicity', 'Precision', 'Generalization']
if not all(col in df.columns for col in features):
    print(f"Error: One or more feature columns {features} missing in CSV file.")
    exit(1)

X = df[features].to_numpy()
y = df['Quality'].to_numpy()

# Check for NaN or infinite values
if np.any(np.isnan(X)) or np.any(np.isinf(X)):
    print("Warning: Found NaN or infinite values in features. Replacing with 0.")
    X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 7: Split data
try:
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42, stratify=y
    )
except ValueError as e:
    print(f"Error during train-test split: {e}")
    print("Class distribution in full dataset:")
    print(pd.Series(y).value_counts())
    print("Falling back to non-stratified split...")
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42
    )
    if len(np.unique(y_train)) < 2:
        print("Error: Training set contains only one class after non-stratified split.")
        print("Training set class distribution:")
        print(pd.Series(y_train).value_counts())
        print("Selecting best combination by Pareto Front.")
        best_combination_idx = select_best_combination(df, thresholds)
        
        best_combination = df.loc[best_combination_idx, 'Combination']
        best_metrics = df.loc[best_combination_idx, ['Fitness', 'Simplicity', 'Precision', 'Generalization']]
        best_silhouette = df.loc[best_combination_idx, 'Silhouette Index']
        best_composite_score = df.loc[best_combination_idx, 'Composite_Score']
        
        print("\nBest Combination (No Classification):")
        print(f"Combination: {best_combination}")
        print(f"Quality: {df.loc[best_combination_idx, 'Quality']}")
        print(f"Metrics: Fitness={best_metrics['Fitness']:.4f}, Simplicity={best_metrics['Simplicity']:.4f}, "
              f"Precision={best_metrics['Precision']:.4f}, Generalization={best_metrics['Generalization']:.4f}")
        print(f"Silhouette Index: {best_silhouette:.4f}")
        print(f"Composite Score: {best_composite_score:.4f}")
        exit(0)

print(f"\nTraining set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print("Training set class distribution:")
print(pd.Series(y_train).value_counts())

# Step 8: Define and fine-tune the model
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1]
}
gb = GradientBoostingClassifier(random_state=42)
cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    gb, param_grid, cv=cv, scoring='f1_weighted', n_jobs=1, error_score='raise'
)

try:
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    print(f"Best parameters: {grid_search.best_params_}")
except Exception as e:
    print(f"Error during GridSearchCV fit: {e}")
    print("Falling back to default GradientBoostingClassifier...")
    try:
        gb = GradientBoostingClassifier(random_state=42)
        gb.fit(X_train, y_train)
        best_model = gb
    except ValueError as ve:
        print(f"Error during fallback fit: {ve}")
        print("Training set class distribution:")
        print(pd.Series(y_train).value_counts())
        print("Selecting best combination by Pareto Front.")
        best_combination_idx = select_best_combination(df, thresholds)
        
        best_combination = df.loc[best_combination_idx, 'Combination']
        best_metrics = df.loc[best_combination_idx, ['Fitness', 'Simplicity', 'Precision', 'Generalization']]
        best_silhouette = df.loc[best_combination_idx, 'Silhouette Index']
        best_composite_score = df.loc[best_combination_idx, 'Composite_Score']
        
        print("\nBest Combination (No Classification):")
        print(f"Combination: {best_combination}")
        print(f"Quality: {df.loc[best_combination_idx, 'Quality']}")
        print(f"Metrics: Fitness={best_metrics['Fitness']:.4f}, Simplicity={best_metrics['Simplicity']:.4f}, "
              f"Precision={best_metrics['Precision']:.4f}, Generalization={best_metrics['Generalization']:.4f}")
        print(f"Silhouette Index: {best_silhouette:.4f}")
        print(f"Composite Score: {best_composite_score:.4f}")
        exit(0)

# Step 9: Evaluate the model
y_pred = best_model.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, zero_division=0))

# Step 10: Predict the best combination
df['Predicted_Quality'] = best_model.predict(scaler.transform(df[features].to_numpy()))
df['Combination'] = df['Method'] + '_' + df['Algorithm']

# Select the best combination using Pareto Front
quality_priority = {'Good': 3, 'Acceptable': 2, 'Poor': 1}
df['Quality_Score'] = df['Predicted_Quality'].map(quality_priority)

best_combination_idx = select_best_combination(df, thresholds)
best_combination = df.loc[best_combination_idx, 'Combination']
best_metrics = df.loc[best_combination_idx, features]
best_silhouette = df.loc[best_combination_idx, 'Silhouette Index']
best_composite_score = df.loc[best_combination_idx, 'Composite_Score']

print("\nBest Combination:")
print(f"Combination: {best_combination}")
print(f"Predicted Quality: {df.loc[best_combination_idx, 'Predicted_Quality']}")
print(f"Metrics: Fitness={best_metrics['Fitness']:.4f}, Simplicity={best_metrics['Simplicity']:.4f}, "
      f"Precision={best_metrics['Precision']:.4f}, Generalization={best_metrics['Generalization']:.4f}")
print(f"Silhouette Index: {best_silhouette:.4f}")
print(f"Composite Score: {best_composite_score:.4f}")